In [ ]:
%matplotlib widget
#%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d 
from helita.vis import rh15d_vis

from scipy.integrate import quadrature
from scipy.integrate import cumtrapz

from pathlib import Path

rhRepoPath = Path("/path/to/rh")


# Working with existing atmosperes' models

```
# There are four 1D models of flares on M-dwarfs:

# radyn_out.F13_dpl_GRID_pt1s.ncdf
# tx.m2F12-37.max2F12_Ec37_d3.ncdf
# tx.m5F12-37.max5F12_Ec37_d3.ncdf
# tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf

# the three "tx" models all have a low cut-off energy of 37 keV and a power index of 3 
# but vary in maximum beam energy: 2e12, 5e12 and 1e13 erg/cm^2/s. 

#The model from Kowalski et al. 2015 is radyn_out.F13_dpl_GRID_pt1s.ncdf.
```



In [ ]:
# for reading the atmospheres files
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = rhRepoPath / "Atmos/"


In [ ]:
# to read xarray.Dataset and see dimentions, coordinates, data variables and attributes 
import xarray
atmos = xarray.open_dataset(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf")
# atmos = xarray.open_dataset(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf")
atmos


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
ed1t=np.transpose(hf['electron_density'][0,:,0,:])

# graphs for temperature, velocity_z and electron density for all time columns

fig, ax=plt.subplots()

n = 50
# n = 52
# n = 189
colors = plt.cm.jet(np.linspace(0,1,n))
for i in range(n):
    # ax.plot(z1t[:,i],np.log10(tg1t[:,i]), color=colors[i])
    # ax.plot(z1t[:,i],(vz1t[:,i]), color=colors[i])
    ax.plot(z1t[:,i],(np.log10(ed1t[:,i])), color=colors[i])


    line = ax.lines[i]

plt.xlabel('Height (Mm)')

# plt.xlim(-0.2,0.5)
# plt.ylim(-1e5,2e5)


# plt.ylabel('lg(Temperature (K))')
# plt.ylabel('Velocity (m/s)')
plt.ylabel('lg(Electron Density (m-3))')
# plt.savefig('temperature')
# plt.savefig('velocity')
plt.savefig('eldensity')



```
# Plotting atmospheres in real height for specific time column
```

In [ ]:
%matplotlib widget


# hf=h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
# hf=h5py.File(dpath_sim+"tx.m5F12-37.max2F12_Ec37_d3.ncdf","r")
hf=h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")

hf.keys()

tg1t=np.transpose(hf['temperature'][0,:,0,:])
vz1t=np.transpose(hf['velocity_z'][0,:,0,:])
ed1t=np.transpose(hf['electron_density'][0,:,0,:])

z1t=np.transpose(hf['z'][0,:,0,:]/1.e6)

fig, axs=plt.subplots()

axs.plot(z1t[:,0],np.log10(tg1t[:,120]),label='12.0 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,75]),label='7.5 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,60]),label='6.0 sec')

axs.plot(z1t[:,0],np.log10(tg1t[:,40]),label='4.0 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,22]),label='2.2 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,12]),label='1.2 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,2]),label='0.2 sec')
axs.plot(z1t[:,0],np.log10(tg1t[:,1]),label='0.1 sec')


# ax.plot(z1t[:,0],np.log10(tg1t[:,0]),'g-',label='0 sec')

# ax.plot(z1t[:,0],vz1t[:,135],label='13.5 sec')
# ax.plot(z1t[:,0],vz1t[:,90],label='9.0 sec')
# ax.plot(z1t[:,0],vz1t[:,60],label='6.0 sec')

# ax.plot(z1t[:,0],vz1t[:,40],label='4.0 sec')
# ax.plot(z1t[:,0],vz1t[:,22],label='2.2 sec')
# ax.plot(z1t[:,0],vz1t[:,12],label='0.2 sec')
# ax.plot(z1t[:,0],vz1t[:,2],label='0.2 sec')

# ax.plot(z1t[:,0],vz1t[:,1],label='0.1 sec')
# ax.plot(z1t[:,0],vz1t[:,0],label='0 sec')

# ax.plot(z1t[:,0],np.log10(ed1t[:,130]),label='13.0 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,100]),label='10.0 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,75]),label='7.5 sec')

# ax.plot(z1t[:,0],np.log10(ed1t[:,40]),label='4.0 sec')
#ax.plot(z1t[:,0],np.log10(ed1t[:,22]),label='2.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,12]),label='1.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,2]),label='0.2 sec')
# ax.plot(z1t[:,0],np.log10(ed1t[:,0]),label='0 sec')




plt.xlabel('Height (Mm)')

plt.ylabel('lg(Temperature (K))')
# plt.ylabel('Velocity (m/s)')
# plt.xlim(-0.2,0.5)
# plt.ylim(-1e5,2e5)
#plt.ylabel('lg(Electron Density ())')
#plt.title('m2F12-37 Atmos’)
# plt.title('txmF13-37 Atmos’)

axs.legend()  # Add a legend.


plt.savefig("temperature_.svg", format="svg")
# plt.savefig("velocity_z_.svg", format="svg")
# plt.savefig("electron_density_.svg", format="svg")


# Create own wavetable file

```
# Writes RH wave file (in xdr format). All wavelengths should be in nm.

# Parameters
# ----------
# start: number
#     Starting wavelength.
# end: number
#     Ending wavelength (non-inclusive)
# step: number
#     Wavelength separation
# outfile: string
#     Name of file to write.
# ewave: 1-D array, optional
#    Array of existing wavelengths. Program will make discard points
#    to make sure no step is enforced using these points too.
# air: boolean, optional
#     If true, will at the end convert the wavelengths into vacuum
#     wavelengths.
```

In [ ]:
def make_wave_file(my_wave, start=20, end=4000, step=5, new_wave=None,
                   ewave=None, air=False):
                   rh15d.make_wave_file('my_wave.wave', ewave=my_waves)

In [ ]:
rh15d.make_wave_file('My_wave.wave', 20, 4000, 5)

In [ ]:

#read exisitng file if needed
MY_wave = rh15d.read_wave_file(rhRepoPath / "Atoms/wave_files/My_wave.wave")


# Tau=1 height

In [ ]:
hf=h5py.File(rhRepoPath / "rh15d/run/output/output_ray.hdf5","r")

hf.keys()
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")



In [ ]:
%matplotlib widget
# plotting tau=1 heights for specific time columns
fig, ax=plt.subplots()

ax.plot(wave[indices],data.ray.tau_one_height[0,0,indices]/1e6, 'k--',label='0 sec')
ax.plot(wave[indices],data.ray.tau_one_height[2,0,indices]/1e6, 'r-',label='0.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[12,0,indices]/1e6, 'r-.',label='1.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[22,0,indices]/1e6, 'b--',label='2.2 sec')
ax.plot(wave[indices],data.ray.tau_one_height[40,0,indices]/1e6, 'b-.',label='4. sec')

ax.legend(loc="upper right") 
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel('Height (Mm)')
plt.title(r'$\tau=1$')
plt.savefig("Tau_one_height_.svg", format="svg")


# Writing ray.input file
```
# Selecting wavelengths for detailed output
# Choosing all wavelenghts range
# Ray input includes all indises
```

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
indices = np.arange(len(data.ray.wavelength))
with open(rhRepoPath / "rh15d/run/ray.input", "w") as f:

    f.write('1.00\n')
    output = str(len(indices))
    for ind in indices:
    
        output += f" {ind}"
    f.write(output)
f.close()

```
#Need to run RH15D again with a new ray.input file 

# Calculating Flux from Intensity(mu)

```
# reading new output
# writing all tau (time-dependant) in one array
```

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
indices = np.arange(len(data.ray.wavelength))
aar = [0,2,12,22,40,60,120] # write down all time columns needed
for a in aar:
    height = data.atmos.height_scale[a, 0].dropna('height')  
    
    index = []
    for i in indices:
        idx = np.argmin(np.abs(data.ray.wavelength.data - data.ray.wavelength[i].data))
        index.append(idx)
    massivTau = []
    for i in indices:
        x = cumtrapz(data.ray.chi[a, 0, :, i].dropna('height'), x=-height)
        x = np.concatenate([[1e-20], x])
        massivTau.append(x)
    np.save(f"massivTau-{a}", massivTau) 


```
# calculating I(mu) for 5 mu
```

In [ ]:
from scipy import integrate
from scipy.interpolate import InterpolatedUnivariateSpline


sourceFunction = data.ray.source_function

mu = [0.0469101,0.230765,0.500000,0.769235,0.953090]
weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]
massivTau = np.load("massivTau-2.npy") #change every time for new massivTau-?.npy

for m in range(5):
    intensityMu = []

    for y in indices:
    
        tmpMu = 0
               
        x = massivTau[y]
        
        tmpMu = integrate.trapz(sourceFunction[2,0,:,y].data * np.exp(-x[:]/mu[m]), x)  #change every time for new sourceFunction[?,0,:,y].data

        intensityMu.append(1/mu[m] * tmpMu)
    np.save(f"intensityMu-{m+1}.npy", intensityMu)

    print(f"Number of elements: {len(intensityMu)}")
    print(f"First element: {intensityMu[0]}")
    print(f"Second element: {intensityMu[1]}")
    print(f"Last element: {intensityMu[1643]}")




In [ ]:
%matplotlib widget

wave = data.ray.wavelength
# graph for all intensities(mu)
ImuLoaded1 = np.load("IntensityMu-1.npy")
ImuLoaded2 = np.load("IntensityMu-2.npy")
ImuLoaded3 = np.load("IntensityMu-3.npy")
ImuLoaded4 = np.load("IntensityMu-4.npy")
ImuLoaded5 = np.load("IntensityMu-5.npy")
fig, ax=plt.subplots()



ax.plot(wave[indices], ImuLoaded1,'k--',label=r'$\mu$1')
ax.plot(wave[indices], ImuLoaded2,'k-',label=r'$\mu$2')
ax.plot(wave[indices], ImuLoaded3,'g-',label=r'$\mu$3')
ax.plot(wave[indices], ImuLoaded4,'r-',label=r'$\mu$4')
ax.plot(wave[indices], ImuLoaded5,'y--',label=r'$\mu$5')

ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Intensity (Stokes I) ($W/(m^2*Hz*sr)$')
#ax.set_yscale('log')
#plt.axis([20, 4000, 0, 8e-8])
plt.savefig("Intencity_.svg", format="svg")

In [ ]:
from scipy import integrate
# calaulating I(mu=1)  
sourceFunction = data.ray.source_function
massivTau = np.load("massivTau-2.npy") #change every time for new massivTau-?.npy

intensityMu =[]

for y in indices: 
    tmpMu = 0
    
    x = massivTau[y]
    
    tmpMu = integrate.trapz(np.log(sourceFunction[2,0,:,y].data * np.exp(-(x[:]))),np.log(x)) #change every time for new sourceFunction[?,0,:,y].data
    
    intensityMu.append(tmpMu)
    

np.save(f"Intensity1Mu0.npy", intensityMu)


In [ ]:
%matplotlib widget
#graphs for I(mu=1) and syntetic disk-centre I, calculated from RH
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")

wave = data.ray.wavelength
ImuLoaded = np.load("Intensity1Mu0.npy")

fig, ax=plt.subplots()

ax.plot(wave[indices],ImuLoaded,'k--',label=r'$\mu$1')

# ax.plot(wave[indices], ImuLoaded5,'r-',label=r'$\mu$0.95') ## for mu=0.95
ax.plot(wave[indices],data.ray.intensity[0,0,indices],'g-.',label='12 sec') #change every time for new data.ray.intensity[?,0,indices]


ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Intensity (Stokes I) ($W/(m^2*Hz*sr)$')
#ax.set_yscale('log')
# plt.axis([20, 4000, 0, 6e-8])
plt.savefig("Intencity_.svg", format="svg")

In [ ]:
# Calculating Flux from I(mu)

fluxesImu =[]
weightMu = [0.118463,0.239314,0.284444,0.239314,0.118463]
mu = [0.0469101,0.230765,0.500000,0.769235,0.953090]
ImuLoaded1 = np.load("IntensityMu-1.npy")
ImuLoaded2 = np.load("IntensityMu-2.npy")
ImuLoaded3 = np.load("IntensityMu-3.npy")
ImuLoaded4 = np.load("IntensityMu-4.npy")
ImuLoaded5 = np.load("IntensityMu-5.npy")
Imuloaded = []
Imuloaded.append(ImuLoaded1)
Imuloaded.append(ImuLoaded2)
Imuloaded.append(ImuLoaded3)
Imuloaded.append(ImuLoaded4)
Imuloaded.append(ImuLoaded5)

for i in indices:
    imu = 0
    for i5 in range(len(weightMu)):
        imu += Imuloaded[i5][i]*weightMu[i5]*mu[i5]
    fluxesImu.append(2*np.pi*imu)
print(len(fluxesImu))


In [ ]:
np.save("fluxesImu12.npy", fluxesImu) #change every time for new fluxesImu?.npy


In [ ]:
%matplotlib widget
# after saving all F, plot them in one graph

fluxesImu1 = np.load("fluxesImu0.npy")
# fluxesImu2 = np.load("fluxesImu1.npy")
fluxesImu2 = np.load("fluxesImu2.npy")
# fluxesImu3 = np.load("fluxesImu6.npy")
# fluxesImu4 = np.load("fluxesImu11.npy")
fluxesImu3 = np.load("fluxesImu12.npy")
fluxesImu4 = np.load("fluxesImu22.npy")
fluxesImu4 = np.load("fluxesImu40.npy")
fluxesImu6 = np.load("fluxesImu60.npy")
fluxesImu7 = np.load("fluxesImu120.npy")



fig, ax=plt.subplots()


wave = data.ray.wavelength
rows = ('0 sec','0.2 sec','1.2 sec','2.2 sec','4 sec','6 sec','12 sec')

colors = plt.cm.tab10(np.linspace(0.3, 1, len(rows)))

ax.plot(wave[indices], fluxesImu1,label='0 sec',color=colors[0],linewidth=0.8)
ax.plot(wave[indices], fluxesImu2,label='0.2 sec',color=colors[1],linewidth=0.8)
ax.plot(wave[indices], fluxesImu3,label='1.2 sec',color=colors[2],linewidth=0.8)
ax.plot(wave[indices], fluxesImu4,label='2.2 sec',color=colors[3],linewidth=0.8)
ax.plot(wave[indices], fluxesImu5,label='4.0 sec',color=colors[4],linewidth=0.8)
ax.plot(wave[indices], fluxesImu6,label='6.0 sec',color=colors[5],linewidth=0.8)
ax.plot(wave[indices], fluxesImu7,label='12.0 sec',color=colors[6],linewidth=0.8)


ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Flux ($W/(m^2*Hz)$')
plt.subplots_adjust(left=0.2, bottom=0.1)
# ax.set_yscale('log')
# plt.axis([20, 4000, 0, 8e-8])
plt.savefig("FluxperHz_.svg", format="svg")
plt.savefig("FluxperHz_")


# plt.xlim(0,2000)
# plt.axis([0, 2000, 0, 1.3e5])
# plt.axis([200, 1000,0, 1e-7])


```
# calculating flux in erg cm-2 s-1 Å-1
```

In [ ]:
const1 = 2.99792458e+21
fluxesImu = np.load("fluxesImu120.npy") #change every time for new fluxesImu?.npy
fluxesSurfaceMu = []

for i in indices:
    fluxesSurfaceMu.append(const1 * fluxesImu[i] /(np.power(wave[i] * 10,2)))

np.save("fluxesSurfaceMu120.npy", fluxesSurfaceMu) #change every time for new fluxesSurfaceMu?.npy

In [ ]:
fluxesSurfaceMu1 = np.load("fluxesSurfaceMu0.npy")
fluxesSurfaceMu2 = np.load("fluxesSurfaceMu2.npy")
fluxesSurfaceMu3 = np.load("fluxesSurfaceMu12.npy")
fluxesSurfaceMu4 = np.load("fluxesSurfaceMu22.npy")
fluxesSurfaceMu5 = np.load("fluxesSurfaceMu40.npy")
fluxesSurfaceMu6 = np.load("fluxesSurfaceMu60.npy")
fluxesSurfaceMu7 = np.load("fluxesSurfaceMu120.npy")

rows = ('0 sec','0.2 sec','1.2 sec','2.2 sec','4 sec','6 sec','12 sec')
wave = data.ray.wavelength


fig, ax=plt.subplots()
colors = plt.cm.Dark2(np.linspace(0, 1, len(rows)))

ax.plot(wave[indices]*10, fluxesSurfaceMu1,label='0 sec',color=colors[0],linewidth=0.8)
ax.plot(wave[indices]*10, fluxesSurfaceMu2,label='0.2 sec',color=colors[1],linewidth=0.8)
ax.plot(wave[indices]*10, fluxesSurfaceMu3,label='1.2 sec',color=colors[2],linewidth=0.8)
ax.plot(wave[indices]*10, fluxesSurfaceMu4,label='2.2 sec',color=colors[3],linewidth=0.8)
ax.plot(wave[indices]*10, fluxesSurfaceMu5,label='4.0 sec',color=colors[4],linewidth=0.8)
ax.plot(wave[indices]*10, fluxesSurfaceMu6,label='6.0 sec',color=colors[5],linewidth=0.8)
ax.plot(wave[indices]*10, fluxesSurfaceMu7,label='12.0 sec',color=colors[6],linewidth=0.8)


ax.legend()
plt.xlabel("Wavelenght, Å")
plt.ylabel("Flux, erg/cm^2/s/Å")
plt.axis([2000, 9000, -1e7, 7e8])
# plt.axis([0, 10000, 0, 4.55e9])

plt.savefig("flux_comparison.svg", format="svg")
plt.savefig("flux_comparison")



```
# calculating total flux and effective temperature
```

In [ ]:
from scipy import integrate
fluxesSurfaceMu = np.load("fluxesSurfaceMu0.npy") # for F in time= 0 sec

x = wave[indices]
fluxesSum = integrate.trapz(fluxesSurfaceMu[:], x * 10)
print("{:5e}".format(fluxesSum))

In [ ]:
# calculating T_eff
sigma = 5.6784e-8
Teff = np.power(fluxesSum*1e-3/sigma, 1/4)
print('{:5e}'.format(Teff))

```
# flux F(Hz) normilized by continuum
# The continuum-normalised spectrum is derived by dividing the computed flux 
# by the computed continuum flux in every spectral bin. 
```

In [ ]:
fluxesSur1 = np.load("fluxesImu0.npy")
fluxesSur2 = np.load("fluxesImu2.npy")
fluxesSur3 = np.load("fluxesImu12.npy")
fluxesSur4 = np.load("fluxesImu22.npy")
fluxesSur5 = np.load("fluxesImu40.npy")
fluxesSur6 = np.load("fluxesImu60.npy")
# fluxesSur7 = np.load("fluxesImu120.npy")

FluxHloaded = []
FluxHloaded.append(fluxesSur1)
FluxHloaded.append(fluxesSur2)
FluxHloaded.append(fluxesSur3)
FluxHloaded.append(fluxesSur4)
FluxHloaded.append(fluxesSur5)
FluxHloaded.append(fluxesSur6)
# FluxHloaded.append(fluxesSur7)

for i in range(1,6):
    normFlux = FluxHloaded[i]/FluxHloaded[0]
    
    np.save(f"normFluxSurCon{i+1}.npy", normFlux)

In [ ]:
# Normalized Intensity Graphs

normFluxCon2 = np.load("normFluxSurCon2.npy")
normFluxCon12 = np.load("normFluxSurCon3.npy")
normFluxCon22 = np.load("normFluxSurCon4.npy")
normFluxCon44 = np.load("normFluxSurCon5.npy")
normFluxCon80 = np.load("normFluxSurCon6.npy")
# normFluxCon120 = np.load("normFluxSurCon7.npy")

fig, ax=plt.subplots()

wave = data.ray.wavelength
rows = ('0.2 sec','1.2 sec','2.2 sec','4 sec','6 sec','12 sec')
colors = plt.cm.tab10(np.linspace(0.5, 1, len(rows)))

# ax.ticklabel_format(useMathText=True)

ax.plot(wave[indices], normFluxCon2,label='0.2 sec',color=colors[0],linewidth=0.8)
ax.plot(wave[indices], normFluxCon12,label='1.2 sec',color=colors[1],linewidth=0.8)
ax.plot(wave[indices], normFluxCon22,label='2.2 sec',color=colors[2],linewidth=0.8)
ax.plot(wave[indices], normFluxCon44,label='4.4 sec',color=colors[3],linewidth=0.8)
ax.plot(wave[indices], normFluxCon80,label='8.0 sec',color=colors[4],linewidth=0.8)
# ax.plot(wave[indices], normFluxCon120,label='12.0 sec',color=colors[5],linewidth=0.8)

plt.subplots_adjust(left=0.2, bottom=0.1)
ax.legend()
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Normalized Flux')
# plt.yticks(values, ["{:.1e}".format(val) for val in values])
# #ax.set_yscale('log')
plt.axis([0, 450, -10000, 2e5])
plt.subplots_adjust(left=0.2, bottom=0.1)

plt.savefig("FluxnormSur_.svg", format="svg")
plt.savefig("Fluxnorm_")

# Calculating Normalized Intesity

In [ ]:
data = rh15d.Rh15dout(rhRepoPath / "rh15d/run/output")
hf=h5py.File(rhRepoPath / "rh15d/run/output/output_ray.hdf5","r")

hf.keys()

wave = data.ray.wavelength

print(data.ray.intensity.shape)


In [ ]:
%matplotlib widget
# Syntetic Disk-centre Intensity RH Graphs
Intencity1 = data.ray.intensity[0,0,:]
Intencity2 = data.ray.intensity[2,0,:]
Intencity3 = data.ray.intensity[12,0,:]
Intencity4 = data.ray.intensity[22,0,:]
Intencity5 = data.ray.intensity[40,0,:]
# Intencity6 = data.ray.intensity[40,0,:]
# Intencity7 = data.ray.intensity[120,0,:]

fig, ax = plt.subplots()
ax.plot(wave[indices],Intencity1,label='0 sec')
ax.plot(wave[indices],Intencity2,label='0.2 sec')
ax.plot(wave[indices],Intencity3,label='1.2 sec')
ax.plot(wave[indices],Intencity4,label='2.2 sec')
ax.plot(wave[indices],Intencity5,label='4.0 sec')
# ax.plot(wave[indices],Intencity6,label='8 sec')
# ax.plot(wave[indices],Intencity7,label='12 sec')

# ax.plot(wave[indices], ImuLoaded5,'b-',label=r'$\mu$5')

plt.legend(loc='best')
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Intensity (Stokes I) ($W/(m^2*Hz*sr)$')
# plt.axis([0, 450, -10000, 6e5])

plt.savefig("Intensitym1_.svg", format="svg")


In [ ]:
# Normalized Intensity Graphs

Intencityloaded = []
Intencityloaded.append(Intencity1)
Intencityloaded.append(Intencity2)
Intencityloaded.append(Intencity3)
Intencityloaded.append(Intencity4)
Intencityloaded.append(Intencity5)
Intencityloaded.append(Intencity6)
# Intencityloaded.append(Intencity7)

for i in range(1,6):
    normIntencity = Intencityloaded[i]/Intencityloaded[0]
    
    np.save(f"normIntencityCon{i+1}.npy", normIntencity)

In [ ]:
%matplotlib widget
# normIntencityCon1 = np.load("normIntencityCon0.npy")
normIntencityCon2 = np.load("normIntencityCon2.npy")
normIntencityCon3 = np.load("normIntencityCon3.npy")
normIntencityCon4 = np.load("normIntencityCon4.npy")
normIntencityCon5 = np.load("normIntencityCon5.npy")
# normIntencityCon6 = np.load("normIntencityCon6.npy")
# normIntencityCon7 = np.load("normIntencityCon7.npy")

fig, ax = plt.subplots()

wave = data.ray.wavelength
rows = ('0.2 sec','1.2 sec','2.2 sec','4 sec','6 sec','12 sec')
colors = plt.cm.tab10(np.linspace(0.3, 0.8, len(rows)))

# ax.ticklabel_format(useMathText=True)

# ax.plot(wave[indices], normFluxCon2,label='0.2 sec',color=colors[0],linewidth=0.8)
# ax.plot(indices,normIntencityCon1,label='0 sec')
ax.plot(wave[indices],normIntencityCon2,label='0.2 sec',color=colors[0],linewidth=0.8)
ax.plot(wave[indices],normIntencityCon3,label='1.2 sec',color=colors[1],linewidth=0.8)
ax.plot(wave[indices],normIntencityCon4,label='2.2 sec',color=colors[2],linewidth=0.8)
ax.plot(wave[indices],normIntencityCon5,label='4.0 sec',color=colors[3],linewidth=0.8)
# ax.plot(wave[indices],normIntencityCon6,label='8 sec',color=colors[4],linewidth=0.8)

# ax.plot(wave[indices],normIntencityCon7,label='12 sec')


plt.legend(loc='best')
plt.xlabel(r'$\lambda$ (nm)')
plt.ylabel(r'Normalized Intensity (Stokes I)')
# plt.yticks(values, ["{:.1e}".format(val) for val in values])

plt.axis([0, 450, -10000, 6e5])
plt.subplots_adjust(left=0.2, bottom=0.1)

plt.savefig("IntensityRH_.svg", format="svg")
plt.savefig("IntensityRH")


# Total Flux in UV bands

In [ ]:
# calculating total flux and effective temperature
# first calculate Flux (lambda), then integrate for choosen wavelengt bands
from scipy import integrate
wave = data.ray.wavelength

const1 = 2.99792458e+21
fluxesSurfaceMu = []
# fluxesImu = np.load("fluxesImu0.npy")
# fluxesImu  = np.load("fluxesImu1.npy")
# fluxesImu  = np.load("fluxesImu6.npy")
# fluxesImu  = np.load("fluxesImu11.npy")
# fluxesImu  = np.load("fluxesImu2.npy")
# fluxesImu  = np.load("fluxesImu12.npy")
# fluxesImu  = np.load("fluxesImu22.npy")
fluxesImu  = np.load("fluxesImu40.npy")
# fluxesImu  = np.load("fluxesImu60.npy")

selectedWave = []
# fluxesSurfaceMu.append(const1 * fluxesImu[0] /(np.power(wave[0] * 10,2)))
# print(fluxesSurfaceMu)
for i in indices:
    v = wave[i].data
    if v > 100 and v < 280:
    # if v > 280 and v < 315:
    # if v > 315 and v < 400:
        fluxesSurfaceMu.append(const1 * fluxesImu[i] /(np.power(wave[i].data * 10,2)))
        selectedWave.append(wave[i])
        print(v)    
fluxesSurfaceMuNP = np.array(fluxesSurfaceMu)
selectedWaveNP = np.array(selectedWave)
print(fluxesSurfaceMuNP.shape, selectedWaveNP.shape)

In [ ]:
fluxesSum = integrate.trapz(fluxesSurfaceMuNP[:], selectedWaveNP*10)
print("{:e}".format(fluxesSum))